### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Fall 2018 ###

# Traveling Tourist Problem: Universal Orlando Theme Parks#

#### Hayden Pilsner (hpilsner@wisc.edu 9074728099)
#### Student 2 (email address and student ID number)
#### Student 3 (email address and student ID number)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation)) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated?) Also give an outline of the rest of the report.

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Feel free to include images if you think it'll be helpful:
![fixit flowchart][flow]

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg

## 2. Mathematical model ##

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$\begin{bmatrix}
      1 & 2 \\
       3 & 4
    \end{bmatrix}
    \begin{bmatrix} x \\ y \end{bmatrix} =
    \begin{bmatrix} 5 \\ 6 \end{bmatrix}$$

And here is an example of an optimization problem in standard form:
$$\begin{aligned}
  \underset{x, y}{\text{minimize}}\qquad& \sum_{i\in A}\left(\sum_{t = 1}^{T} \sum_{j\in A} tx_{ij}^{t} - \sum_{t = 1}^{T} ty_{i}^{t} \right)\\
    \text{subject to:}\qquad& \sum_{t = 1}^{T} \sum_{j\in A} x_{ij}^{t} = 1 && \forall i \in A\\
    & \sum_{t = 1}^{T} \sum_{i \in A} x_{ij}^{t} = 1 && \forall j \in A \\
    & \sum_{t = 1}^{T} y_{i}^{t} = 1 && \forall i \in A \\
    & \sum_{i \in A} y_{i}^{t} \le 1 && \forall t \in \{ 1, \dots , T \}\\
    & \sum_{j\in A} x_{ij}^{t} \le \sum_{k = 0}^{t} y_{i}^{k} && \forall i \in A, \quad \forall t \in \{ 1, \dots , T \}\\
    & y_{i}^{t} w_{i}^{t} \le \gamma \sum_{k = t}^{T} \sum_{j\in A} kx_{ij}^{k} - ty_{i}^{t} && \forall i \in A, \quad \forall t \in \{ 1, \dots , T \} \\
    & x_{ij}^{t} \in \{0, 1\} && \forall i,j \in A, \quad \forall t \in \{ 1, \dots , T \} \\
    & y_{i}^{t} \in \{0, 1\} && \forall i \in A, \quad \forall t \in \{ 1, \dots , T \}
    \end{aligned}$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

### Minimizing Wait Time: ###

In [2]:
using JuMP, Gurobi, NamedArrays
s = 40    # number of time s
z = 15    # interval of time s
α = 15
# import wait times
raw_wait = readcsv("wait_times.csv")
(u,v) = size(raw_wait)

n_waitTimes = 2:v      # columns containing waitTimes
n_rides = 2:u          # rows containing rides names

waitTimes = raw_wait[1,n_waitTimes][:]   
rides = raw_wait[n_rides,1][:]           

w = NamedArray( raw_wait[n_rides,n_waitTimes], (rides,waitTimes), ("Rides","Wait Times") );
r = u - 1    # number of rides

m = Model(solver=GurobiSolver(OutputFlag=0))

@variable(m, x[1:r, 1:r, 1:s], Bin)    # departures from i to j at time t
@variable(m, y[1:r, 1:s], Bin)    # arrivals at i at time t

@constraint(m, c1[i in 1:r], sum(sum(x[i,j,t] for j in 1:r) for t in 1:s) == 1)    # can only leave ride once
@constraint(m, c2[i in 1:r], sum(sum(x[j,i,t] for j in 1:r) for t in 1:s) == 1)    # can only come from one ride
@constraint(m, c3[i in 1:r], sum(y[i,t] for t in 1:s) == 1)    # ride each ride once and only once
@constraint(m, c4[t in 1:s], sum(y[i,t] for i in 1:r) <= 1)   # can only arrive at one ride at one time
# you cannot leave i before you arrive at i
@constraint(m, c5[i in 1:r, t in 1:s], sum(x[i,j,t] for j in 1:r) <= sum(y[i,k] for k in 1:t))
# difference between departure and arrival time must be at least the wait time at i, if arriving at i at time t
@constraint(m, c6[i in 1:r, t in 1:s], z*(sum(sum(k*x[i,j,k] for j in 1:r) for k in t:s) - t*y[i,t]) >= y[i,t] * w[i,t])

# wait time
@expression(m, wait, sum(sum(sum(z*t*x[i,j,t] for j in 1:r) for t in 1:s) - sum(z*t*y[i,t] for t in 1:s) for i in 1:r))
@objective(m, Min, wait)    # minimize wait time

solve(m)
println("Min wait time: ", getobjectivevalue(m))
sched_data = Array{String}(r, 2)
sched_headers = ["Arrival Time", "Wait (min)"]

opty = getvalue(y)
optx = getvalue(x)
order = 1

ride_order = copy(rides)
for i in 1:s
    for j in 1:r
        if (opty[j,i] == 1.0)
            ride_order[order] = rides[j]
            sched_data[order,1] = waitTimes[i]
            sched_data[order,2] = string(w[j,i])
            order += 1
        end
    end
end

schedule = NamedArray( sched_data[1:r,1:2], (ride_order, sched_headers), ("Rides","") );
println(schedule)

Academic license - for non-commercial use only
Min wait time: 90.0
6×2 Named Array{String,2}
                            Rides ╲  │ Arrival Time    Wait (min)
─────────────────────────────────────┼───────────────────────────
"Fast & Furious - Supercharged"      │    "9:15 AM"          "12"
"Harry Potter Escape from Gringotts" │   "10:00 AM"          "14"
"MEN IN BLACK Alien Attack"          │   "10:45 AM"           "6"
"The Simpsons Ride"                  │   "11:15 AM"           "8"
"Shrek 4-D"                          │    "2:45 PM"           "7"
"Transformers: The Ride 3D"          │    "6:30 PM"           "5"


In [2]:
function import_walk(filename)        # import walk times
    raw_walk = readcsv(filename)
    r = size(raw_walk)[1] - 1 # number of rides
    return raw_walk[2:r+1, 2:r+1]
end

function import_wait(filename, interval_len)    # import wait times
    raw_wait = readcsv(filename)
    r = length(raw_wait[:,1]) - 1
    slots = size(raw_wait[1, :])[1] - 1
    w = Matrix(0,slots*interval_len)
    raw_wait = raw_wait[2:r+1,2:slots+1]
    for i in 1:r
        w_row = []
        for j in 1:slots
            for k in 1:interval_len
                push!(w_row, raw_wait[i,j])
            end
        end
        w = [w ; transpose(w_row)]
    end
    return w
end

import_wait (generic function with 1 method)

In [2]:
using JuMP, Gurobi

function shortest_distance(c)
    r = length(c[1,:])
    
    m = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(m, x[1:r, 1:r], Bin)                                      # decision matrix going from ride i to j
    # TSP constraints
    @constraint(m, c1[j in 1:r], sum( x[i,j] for i in 1:r ) == 1)       # one out-edge
    @constraint(m, c2[i in 1:r], sum( x[i,j] for j in 1:r ) == 1)       # one in-edge
    @constraint(m, c3[i in 1:r], x[i,i] == 0 )                          # no self-loops

    # Miller-Tucker-Zemlin variables and constraints to eliminate subtours
    @variable(m, u[1:r])
    @constraint(m, c4[i in 1:r, j in 2:r], u[i] - u[j] + r*x[i,j] <= r-1 )

    @expression(m, walk, sum(x[i,j]*c[i,j] for i in 1:r, j in 1:r))
    @objective(m, Min, walk)   # minimize total cost

    solve(m)
    println(getobjectivevalue(m))
    optx = getvalue(x)
    println(getvalue(x))
end

c = import_walk("walk_times.csv")    # symmetric walk time matrix
w = import_wait("wait_times.csv", 15)     # asymmetric temporal wait time matrix (wait time at ride i at minute t)
shortest_distance(c)

Academic license - for non-commercial use only
26.0
[-0.0 0.0 1.0 0.0 0.0 0.0; 0.0 -0.0 0.0 1.0 0.0 0.0; 0.0 1.0 -0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.0 1.0 0.0; 0.0 0.0 0.0 0.0 -0.0 1.0; 1.0 0.0 0.0 0.0 0.0 -0.0]


In [7]:
using JuMP, Gurobi

function optimal_theme_park_tour(walk_λ, wait_λ, β, T, c, w)
    r = length(c[1,:])

    m = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(m, x[1:r, 1:r], Bin)                                      # decision matrix going from ride i to j
    @variable(m, a[1:r, 1:T], Bin)                                      # decision matrix riding ride i at minute t
    # TSP constraints
    @constraint(m, c1[j in 1:r], sum( x[i,j] for i in 1:r ) == 1)       # one out-edge
    @constraint(m, c2[i in 1:r], sum( x[i,j] for j in 1:r ) == 1)       # one in-edge
    @constraint(m, c3[i in 1:r], x[i,i] == 0 )                          # no self-loops

    # Miller-Tucker-Zemlin variables and constraints to eliminate multiple subtours
    @variable(m, u[1:r])
    @constraint(m, c4[i in 1:r, j in 2:r], u[i] - u[j] + r*x[i,j] <= r-1 )
    
    if (wait_λ > 0)
        @constraint(m, c5[i in 1:r], (1 - x[i,1])*sum(a[i,t]*(β + t + w[i,t]) for t in 1:T) + sum(x[i,j]*c[i,j] for j in 2:r) == sum(t*sum(a[j,t]*x[i,j] for j in 2:r) for t in 1:T))
        @constraint(m, c6[i in 1:r], sum(a[i,t] for t in 1:T) == 1)
        @constraint(m, c7, a[1,1] == 1)
    end

    @expression(m, walk, sum(x[i,j]*c[i,j] for i in 1:r, j in 1:r))    # walk time
    @expression(m, wait, sum(sum(a[i,t]*w[i,t] for t in 1:T) for i in 1:r))    # wait time
    @objective(m, Min, walk_λ*walk + wait_λ*wait)   # minimize total walk and wait time

    solve(m)
    println(getobjectivevalue(m))
    optx = getvalue(x)
    opta = getvalue(a)
    for i in 1:r
        print(sum(t*opta[i,t] for t in 1:T), " ")
    end
    println("")
end

# Universal Studios Park
println("Universal Studios:")
c = import_walk("studios_walk.csv")    # symmetric walk time matrix
w = import_wait("studios_wait.csv", 15)     # asymmetric temporal wait time matrix (wait time at ride i at minute t)
optimal_theme_park_tour(1, 1, 10, 150, c, w)

# Islands of Adventure Park
println("Islands of Adventure:")
c = import_walk("ioa_walk.csv")    # symmetric walk time matrix
w = import_wait("ioa_wait.csv", 15)     # asymmetric temporal wait time matrix (wait time at ride i at minute t)
optimal_theme_park_tour(1, 1, 10, 150, c, w)

Universal Studios:
Academic license - for non-commercial use only
70.99999999999987
1.0 18.00000000000006 41.99999999999999 64.0 121.99999999999727 83.99999999999683 105.99999999999956 
Islands of Adventure:
Academic license - for non-commercial use only
61.0
1.0 15.0 77.0 96.0 59.0 36.0 111.0 


## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

 Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are          | Cool  |
| ------------- |:-------------| -----:|
| col 3 is      |right-aligned |\$1600 |
|  colons       | align columns|  \$12 |
| zebra stripes |    are neat  |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.